In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn("ingestion_date", current_timestamp())
  return output_df

In [0]:
def re_arrange_partition_column(input_df, partition_column):
  column_list = []
  for column_name in input_df.schema.names:
    if column_name != partition_column:
      column_list.append(column_name)
  column_list.append(partition_column)
  output_df = input_df.select(column_list)
  return output_df

In [0]:
def overwrite_function(input_df,database_name, table_name, Partition_column):
    output_df = re_arrange_partition_column(input_df,Partition_column)
    spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

    if (spark._jsparkSession.catalog().tableExists(f"{database_name}.{table_name}")):
      output_df.write.mode("overwrite").insertInto(f"{database_name}.{table_name}")
    else:
      output_df.write.mode("overwrite").partitionBy(Partition_column).format("parquet").saveAsTable(f"{database_name}.{table_name}")

In [0]:
def df_column_to_list(input_df, column_name):
    df_row_list = input_df.select(column_name) \
                        .distinct() \
                        .collect()
    column_value_list = [row[column_name] for row in df_row_list]
    return column_value_list
